# LZW Encoding

This notebook shows the use of the `lzw_encode` and `lzw_decode` methods in `slz`. There is both a functional implementation and an object-oriented (stateful) implementation.


### Encoding
The encoding function `lzw_decode` has the following signature


```python
def lzw_encode(data: Union[str, bytes]) -> str:
```

In reality, the `str` return is really just a byte stream returned from the encoder. It almost always makes sense to do `b = lzw_encode(data).encode("utf-8")`. The reason the return type is `str` rather than `bytes` is to do with how bytes work in Python and what is possible with `pybind11`. See for example [https://github.com/pybind/pybind11/issues/1236]. 

The signature in the underlying implementation is 

```c++
std::stringstream lzw_encode(const std::string_view input);
```

Because the return type is strictly `str` we need to explicitly convert to `bytes` if a byte stream is required by doing ```enc_out.encode("utf-8")```.

## Encoding format.

The output of the encoder is a byte stream containing the compressed bytes. To be able to recover the stream information correctly when decoding some header information is prepended to the data. The data stream consists of
symbols that vary in size from 2-4 bytes per symbol. At the start of the stream all symbols are 2 bytes. As the prefix tree gets longer it becomes possible to gain more compression by using a larger symbol that represents a longer prefix. At some point the stream will contain symbols that are all 3 bytes long, and then even later all symbols will be 4 bytes long. When decoding the stream we need to know at which point the size of the symbols changes. We encode this information in a 12 byte header at the start of the stream

The header format is 

- (__4 bytes__) - offset in stream where the first 24-bit code is. If this is zero there are no 24-bit codes
- (__4 bytes__) - offset in stream where the first 32-bit code is. If this is zero there are no 32-bit codes.
- (__4 bytes__) - total number of codes in the stream.

The byte at offset 12 is the first byte in the data stream and is part of a 2-byte symbol.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# TODO: hack - need to pip install package to avoid this?
import os
os.chdir("..")    # cant load slz as a module from the notebooks directory

In [3]:
from slz import lzw_encode      # import the functional encoder

# We start with the example from the unit test 
test_input = "babaabaaa"

encoded = lzw_encode(test_input)
enc_bytes = encoded.encode("utf-8")

print(f"Length of input string  : {len(test_input.encode('utf-8'))}")
print(f"Length of output string : {len(enc_bytes)}")
print(f"bytes: {enc_bytes}")

Length of input string  : 9
Length of output string : 24
bytes: b'\x00\x00\x00\x00\x00\x00\x00\x00\x05\x01\x00\x00b\x00a\x00\x00\x01\x01\x01a\x00\x04\x01'


Note that the output is longer than the input. This is often the case in general with compression - significant compression is achieved in the limit but often not for small inputs. In this specific case though some of the overhead is in the header itself. The first 12 bytes provide information for decoding. In this example the header is longer than the original string, and actually contains relatively little information. If we skip the first 12 bytes 

In [4]:
print(f"bytes: {enc_bytes[11:]}")

bytes: b'\x00b\x00a\x00\x00\x01\x01\x01a\x00\x04\x01'


We can decode a stream with this header using `lzw_decode`. This accepts a byte stream that is expected to have the header format described above, followed by a byte stream of compressed bytes. The signature for `lzw_decode` is

```python
def lzw_decode(data: bytes) -> str:
```

The underlying representation accepts a `std::stringstream` but is wrapped to accept a `const std::string&`. The function accepts `str` or `bytes`, but a `str` formatted for display may not preserve the header information correctly. 

```c++
std::string lzw_decode(const std::string& data)
```

In [5]:
from slz import lzw_decode

dec_out = lzw_decode(enc_bytes)

print(f"Length of input string  : {len(enc_bytes)}")
print(f"Length of output string : {len(dec_out)}")
print(f"str: {dec_out}")

Length of input string  : 24
Length of output string : 9
str: babaabaaa


### Encoding a stream from a file 

Notice that the compression ration on small inputs is very poor. At minimum the size of the stream needs to be logn enoguh to amortize the length of the header. Additionally, we tend to get better compression performance when we have a deep prefix tree. Constructing a deep prefix tree requires us to consume more symbols. In the following cells we explore compressing longer streams.

To avoid getting psy-op'd on copyright issues we use the collected works of Shakespear from Project Gutenberg.

In [37]:
input_filename = "test/shakespear.txt"

with open(input_filename, "r") as fp:
    text = fp.read()
    #text = "".join(fp.readlines())
    
substr = text[1024:2048]
print(type(substr))
print(substr)
print(len(substr))

<class 'str'>
 COMEDY OF ERRORS
    THE TRAGEDY OF CORIOLANUS
    CYMBELINE
    THE TRAGEDY OF HAMLET, PRINCE OF DENMARK
    THE FIRST PART OF KING HENRY THE FOURTH
    THE SECOND PART OF KING HENRY THE FOURTH
    THE LIFE OF KING HENRY THE FIFTH
    THE FIRST PART OF HENRY THE SIXTH
    THE SECOND PART OF KING HENRY THE SIXTH
    THE THIRD PART OF KING HENRY THE SIXTH
    KING HENRY THE EIGHTH
    THE LIFE AND DEATH OF KING JOHN
    THE TRAGEDY OF JULIUS CAESAR
    THE TRAGEDY OF KING LEAR
    LOVE’S LABOUR’S LOST
    THE TRAGEDY OF MACBETH
    MEASURE FOR MEASURE
    THE MERCHANT OF VENICE
    THE MERRY WIVES OF WINDSOR
    A MIDSUMMER NIGHT’S DREAM
    MUCH ADO ABOUT NOTHING
    THE TRAGEDY OF OTHELLO, THE MOOR OF VENICE
    PERICLES, PRINCE OF TYRE
    KING RICHARD THE SECOND
    KING RICHARD THE THIRD
    THE TRAGEDY OF ROMEO AND JULIET
    THE TAMING OF THE SHREW
    THE TEMPEST
    THE LIFE OF TIMON OF ATHENS
    THE TRAGEDY OF TITUS ANDRONICUS
    TROILUS AND CRESSIDA
    TWELF

We encode the string using `lzw_encode`.

In [38]:

#enc = lzw_encode(str(substr))
enc = lzw_encode(substr.encode("utf-8"))


print(len(enc))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 8: invalid continuation byte